# Running end-to-end

The easiest way to run Isofit is through the built-in end-to-end correction pipeline we call "Apply OE". Apply OE is run via a command line interface (CLI) tool accessed with a terminal call:

 `isofit apply_oe --help`

 The function docstring is printed when this command is run within a terminal window, and acts as a guide for how to run the function.

 ```
isofit apply_oe --help

Usage: isofit apply_oe [OPTIONS] INPUT_RADIANCE INPUT_LOC INPUT_OBS
                       WORKING_DIRECTORY SENSOR

  Applies OE over a flightline using a radiative transfer engine. This
  executes ISOFIT in a generalized way, accounting for the types of variation
  that might be considered typical.

  Observation (obs) and location (loc) files are used to determine appropriate
  geometry lookup tables and provide a heuristic means of determining
  atmospheric water ranges.

  Parameters
  ----------
  input_radiance : str
      Radiance data cube. Expected to be ENVI format
  input_loc : str
      Location data cube of shape (Lon, Lat, Elevation). Expected to be ENVI format
  input_obs : str
      Observation data cube of shape:
          (path length, to-sensor azimuth, to-sensor zenith,
          to-sun azimuth, to-sun zenith, phase,
          slope, aspect, cosine i, UTC time)
      Expected to be ENVI format
  working_directory : str
      Directory to stage multiple outputs, will contain subdirectories
  sensor : str
      The sensor used for acquisition, will be used to set noise and datetime
      settings
  surface_path : str
      Path to surface model or json dict of surface model configuration
  copy_input_files : bool, default=False
      Flag to choose to copy input_radiance, input_loc, and input_obs locally into
      the working_directory
  modtran_path : str, default=None
      Location of MODTRAN utility. Alternately set with `MODTRAN_DIR` environment
      variable
  wavelength_path : str, default=None
      Location to get wavelength information from, if not specified the radiance
      header will be used
  surface_category : str, default="multicomponent_surface"
      The type of ISOFIT surface priors to use.  Default is multicomponent_surface
  aerosol_climatology_path : str, default=None
      Specific aerosol climatology information to use in MODTRAN
  rdn_factors_path : str, default=None
      Specify a radiometric correction factor, if desired
  atmosphere_type : str, default="ATM_MIDLAT_SUMMER"
      Atmospheric profile to be used for MODTRAN simulations.  Unused for other
      radiative transfer models.
  channelized_uncertainty_path : str, default=None
      Path to a channelized uncertainty file
  model_discrepancy_path : str, default=None
      Modifies S_eps in the OE formalism as the Gamma additive term, as:
      S_eps = Sy + Kb.dot(self.Sb).dot(Kb.T) + Gamma
  lut_config_file : str, default=None
      Path to a look up table configuration file, which will override defaults
      choices
  multiple_restarts : bool, default=False
      Use multiple initial starting poitns for each OE ptimization run, using
      the corners of the atmospheric variables as starting points.  This gives
      a more robust, albeit more expensive, solution.
  logging_level : str, default="INFO"
      Logging level with which to run ISOFIT
  log_file : str, default=None
      File path to write ISOFIT logs to
  n_cores : int, default=1
      Number of cores to run ISOFIT with. Substantial parallelism is available, and
      full runs will be very slow in serial. Suggested to max this out on the
      available system
  presolve : int, default=False
      Flag to use a presolve mode to estimate the available atmospheric water range.
      Runs a preliminary inversion over the image with a 1-D LUT of water vapor, and
      uses the resulting range (slightly expanded) to bound determine the full LUT.
      Advisable to only use with small cubes or in concert with the empirical_line
      setting, or a significant speed penalty will be incurred
  empirical_line : bool, default=False
      Use an empirical line interpolation to run full inversions over only a subset
      of pixels, determined using a SLIC superpixel segmentation, and use a KDTREE of
      local solutions to interpolate radiance->reflectance. Generally a good option
      if not trying to analyze the atmospheric state at fine scale resolution.
      Mutually exclusive with analytical_line
  analytical_line : bool, default=False
      Use an analytical solution to the fixed atmospheric state to solve for each
      pixel.  Starts by running a full OE retrieval on each SLIC superpixel, then
      interpolates the atmospheric state to each pixel, and closes with the
      analytical solution.
      Mutually exclusive with empirical_line
  ray_temp_dir : str, default="/tmp/ray"
      Location of temporary directory for ray parallelization engine
  emulator_base : str, default=None
      Location of emulator base path. Point this at the model folder (or h5 file) of
      sRTMnet to use the emulator instead of MODTRAN. An additional file with the
      same basename and the extention _aux.npz must accompany
      e.g. /path/to/emulator.h5 /path/to/emulator_aux.npz
  segmentation_size : int, default=40
      If empirical_line is enabled, sets the size of segments to construct
  num_neighbors : list[int], default=[]
      Forced number of neighbors for empirical line extrapolation - overides default
      set from segmentation_size parameter
  atm_sigma : list[int], default=[2]
      A list of smoothing factors to use during the atmospheric interpolation, one
      for each atmospheric parameter (or broadcast to all if only one is provided).
      Only used with the analytical line.
  pressure_elevation : bool, default=False
      Flag to retrieve elevation
  prebuilt_lut : str, default=None
      Use this pre-constructed look up table for all retrievals. Must be an
      ISOFIT-compatible RTE NetCDF
  no_min_lut_spacing : bool, default=False
      Don't allow the LUTConfig to remove a LUT dimension because of minimal spacing.
  inversion_windows : list[float], default=None
      Override the default inversion windows.  Will supercede any sensor specific
      defaults that are in place.
      Must be in 2-item tuples
  config_only : bool, default=False
      Generates the configuration then exits before execution. If presolve is
      enabled, that run will still occur.
  interpolate_bad_rdn : bool, default=False
      Flag to perform a per-pixel interpolation across no-data and NaN data bands.
      Does not interpolate vectors that are entire no-data or NaN, only partial.
      Currently only designed for wavelength interpolation on spectra.
      Does NOT do any spatial interpolation
  interpolate_inplace : bool, default=False
      Flag to tell interpolation to work on the file in place, or generate a
      new interpolated rdn file. The location of the new file will be in the
      "input" directory within the working directory.

  References
  ----------
  D.R. Thompson, A. Braverman,P.G. Brodrick, A. Candela, N. Carbon, R.N. Clark,D. Connelly, R.O. Green, R.F.
  Kokaly, L. Li, N. Mahowald, R.L. Miller, G.S. Okin, T.H.Painter, G.A. Swayze, M. Turmon, J. Susilouto, and
  D.S. Wettergreen. Quantifying Uncertainty for Remote Spectroscopy of Surface Composition. Remote Sensing of
  Environment, 2020. doi: https://doi.org/10.1016/j.rse.2020.111898.

  sRTMnet emulator:
  P.G. Brodrick, D.R. Thompson, J.E. Fahlen, M.L. Eastwood, C.M. Sarture, S.R. Lundeen, W. Olson-Duvall,
  N. Carmon, and R.O. Green. Generalized radiative transfer emulation for imaging spectroscopy reflectance
  retrievals. Remote Sensing of Environment, 261:112476, 2021.doi: 10.1016/j.rse.2021.112476.

Options:
  -sp, --surface_path TEXT        [required]
  --copy_input_files
  --modtran_path TEXT
  --wavelength_path TEXT
  --surface_category TEXT
  --aerosol_climatology_path TEXT
  --rdn_factors_path TEXT
  --atmosphere_type TEXT
  --channelized_uncertainty_path TEXT
  --model_discrepancy_path TEXT
  --lut_config_file TEXT
  --multiple_restarts
  --logging_level TEXT
  --log_file TEXT
  --n_cores INTEGER
  --presolve
  --empirical_line
  --analytical_line
  --ray_temp_dir TEXT
  --emulator_base TEXT
  --segmentation_size INTEGER
  -nn, --num_neighbors INTEGER
  -as, --atm_sigma FLOAT
  --pressure_elevation
  --prebuilt_lut TEXT
  --no_min_lut_spacing
  --inversion_windows FLOAT...
  --config_only
  --interpolate_bad_rdn
  --interpolate_inplace
  --debug-args                    Prints the arguments list without executing
                                  the command
  --profile TEXT
  --help                          Show this message and exit.
  ```

There are a large number of parameters for the Apply OE function, but most are optional. Many have been added to accomodate unique run cases and help software development. The important inputs are the non-optional arguments: 

`INPUT_RADIANCE INPUT_LOC INPUT_OBS WORKING_DIRECTORY SENSOR`

which must be entered in the specified order. Descriptions of each are found in the docstring printed above. It is important to note that the `INPUT_RADIANCE`, `INPUT_LOC`, and `INPUT_OBS` must be at the same row column dimensions.

The Apply OE function uses default parameters for many of the optional parameters (e.g. `--modtran_path`, `--pressure_elevation`). Most of the default behavior should run without issue with a properly set-up `~/.isofit/isofit.ini` file. For example, Apply OE defaults to using the sRTMnet radiative transfer engine, and will use the path provided in the .ini file if none is given.